# MATCH OUTCOME ANALYSES & PREDICTIONS

## Author

Saban Akay, 14045645

## Introduction

### Background Information

### Data Description

The Europian Soccer Dataset is a relational database of soccer matches, players, and teams. It contains data for soccer matches in eleven European countries from 2008 to 2016. The database contains;
- +25,000 matches
- +10,000 players
- 11 European Countries with their lead championship
- Seasons 2008 to 2016
- Players and Teams' attributes* sourced from EA Sports' FIFA video game series, including the weekly updates
- Team line up with squad formation (X, Y coordinates)
- Betting odds from up to 10 providers
- Detailed match events (goal types, possession, corner, cross, fouls, cards etc…) for +10,000 matchesThe data is stored in a SQLite database.

The database is provided by [Hugomathien](https://www.kaggle.com/hugomathien/soccer) on Kaggle.

### Problem Statement

In this notebook, I will try to answer the following questions:

1. What is the most common match outcome?
2. What is the relationship between the match outcome and the team attributes?
3. What is the relationship between the match outcome and the player attributes?
4. What is the relationship between the match outcome and the betting odds?
5. Which relationship is the strongest predictor of the match outcome?


### Methodology

The methodology of this notebook is as follows:

1. [Setup](#Setup)
    - [Import Libraries](#Import-Libraries)
    - [Ultilities](#Ultilities)
2. [Data Preparation](#Data-Preparation)
    - [Data Collection](#Data-Collection)
    - [Data Cleaning](#Data-Cleaning)
    - [Data Transformation](#Data-Transformation)
    - [Data Integration](#Data-Integration)
    - [Feature Engineering](#Feature-Engineering)
3. [Data Analysis](#Data-Analysis)
    - [Exploratory Data Analysis](#Exploratory-Data-Analysis)
    - [Descriptive Statistics](#Descriptive-Statistics)
    - [Data Visualization](#Data-Visualization)
    - [Identifying Patterns and Insights](#Identifying-Patterns-and-Insights)
4. [Prediction](#Prediction)
    - [Model Definition](#Model-Definition)
    - [Model Training](#Model-Training)
    - [Model Testing](#Model-Testing)
    - [Model Evaluation](#Model-Evaluation)
    - [Model Selection](#Model-Selection)
    - [Model Tuning](#Model-Tuning)
5. [Conclusion](#Conclusion)
    - [Discussion](#Discussion)
    - [Limitations](#Limitations)
    - [Future Work](#Future-Work)

## Setup

### Import Libraries

In [125]:
# Import required libraries
import sqlite3 as db
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from time import time
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import make_scorer
from sklearn.decomposition import PCA, FastICA
from sklearn.pipeline import Pipeline
import warnings
import tempfile
from joblib import Memory
import datetime

### Ultilities

In [126]:
warnings.filterwarnings("ignore")

In [127]:
# seed for reproducibility
Seed = 42

In [128]:
# Create a temporary folder to store the transformers of the pipeline
location = tempfile.gettempdir()
memory = Memory(location=location, verbose=-1)

In [129]:
# amount of data to be used (for run time purposes, 100% = 1, 50% = 0.5, etc.)
run = 0.02
n = 0.3

In [130]:
relations = ['team', 'player', 'betting']

## Data Preparation

In this section, we will load and clean the data from the European Soccer Database. 
This may include handling missing values, removing irrelevant columns, and merging different tables of the database.

### Data Collection

In [131]:
# explore the tables in the database
conn = db.connect('../data/database.sqlite')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('sqlite_sequence',), ('Player_Attributes',), ('Player',), ('Match',), ('League',), ('Country',), ('Team',), ('Team_Attributes',)]


In [132]:
# load the data
with db.connect('../data/database.sqlite') as conn:
    matches = pd.read_sql_query('SELECT * FROM Match', conn)
    teams = pd.read_sql_query('SELECT * FROM Team', conn)
    leagues = pd.read_sql_query('SELECT * FROM League', conn)
    countries = pd.read_sql_query('SELECT * FROM Country', conn)
    players = pd.read_sql_query('SELECT * FROM Player', conn)
    player_attributes = pd.read_sql_query('SELECT * FROM Player_Attributes', conn)
    team_attributes = pd.read_sql_query('SELECT * FROM Team_Attributes', conn)


In [133]:
# explore the columns in matches table
print(f'{matches.columns.values} \t {matches.shape}')

['id' 'country_id' 'league_id' 'season' 'stage' 'date' 'match_api_id'
 'home_team_api_id' 'away_team_api_id' 'home_team_goal' 'away_team_goal'
 'home_player_X1' 'home_player_X2' 'home_player_X3' 'home_player_X4'
 'home_player_X5' 'home_player_X6' 'home_player_X7' 'home_player_X8'
 'home_player_X9' 'home_player_X10' 'home_player_X11' 'away_player_X1'
 'away_player_X2' 'away_player_X3' 'away_player_X4' 'away_player_X5'
 'away_player_X6' 'away_player_X7' 'away_player_X8' 'away_player_X9'
 'away_player_X10' 'away_player_X11' 'home_player_Y1' 'home_player_Y2'
 'home_player_Y3' 'home_player_Y4' 'home_player_Y5' 'home_player_Y6'
 'home_player_Y7' 'home_player_Y8' 'home_player_Y9' 'home_player_Y10'
 'home_player_Y11' 'away_player_Y1' 'away_player_Y2' 'away_player_Y3'
 'away_player_Y4' 'away_player_Y5' 'away_player_Y6' 'away_player_Y7'
 'away_player_Y8' 'away_player_Y9' 'away_player_Y10' 'away_player_Y11'
 'home_player_1' 'home_player_2' 'home_player_3' 'home_player_4'
 'home_player_5' 'home_pl

In [134]:
# explore the columns in countries table
print(f'{countries.columns.values} \t {countries.shape}')

['id' 'name'] 	 (11, 2)


In [135]:
# explore the columns in leagues table
print(f'{leagues.columns.values} \t {leagues.shape}')

['id' 'country_id' 'name'] 	 (11, 3)


In [136]:
# explore the columns in players table
print(f'{players.columns.values} \t {players.shape}')

['id' 'player_api_id' 'player_name' 'player_fifa_api_id' 'birthday'
 'height' 'weight'] 	 (11060, 7)


In [137]:
# explore the columns in player_attributes table
print(f'{player_attributes.columns.values} \t {player_attributes.shape}')

['id' 'player_fifa_api_id' 'player_api_id' 'date' 'overall_rating'
 'potential' 'preferred_foot' 'attacking_work_rate' 'defensive_work_rate'
 'crossing' 'finishing' 'heading_accuracy' 'short_passing' 'volleys'
 'dribbling' 'curve' 'free_kick_accuracy' 'long_passing' 'ball_control'
 'acceleration' 'sprint_speed' 'agility' 'reactions' 'balance'
 'shot_power' 'jumping' 'stamina' 'strength' 'long_shots' 'aggression'
 'interceptions' 'positioning' 'vision' 'penalties' 'marking'
 'standing_tackle' 'sliding_tackle' 'gk_diving' 'gk_handling' 'gk_kicking'
 'gk_positioning' 'gk_reflexes'] 	 (183978, 42)


In [138]:
# explore the columns in teams table
print(f'{teams.columns.values} \t {teams.shape}')

['id' 'team_api_id' 'team_fifa_api_id' 'team_long_name' 'team_short_name'] 	 (299, 5)


In [139]:
# explore the columns in team_attributes table
print(f'{team_attributes.columns.values} \t {team_attributes.shape}')

['id' 'team_fifa_api_id' 'team_api_id' 'date' 'buildUpPlaySpeed'
 'buildUpPlaySpeedClass' 'buildUpPlayDribbling'
 'buildUpPlayDribblingClass' 'buildUpPlayPassing'
 'buildUpPlayPassingClass' 'buildUpPlayPositioningClass'
 'chanceCreationPassing' 'chanceCreationPassingClass'
 'chanceCreationCrossing' 'chanceCreationCrossingClass'
 'chanceCreationShooting' 'chanceCreationShootingClass'
 'chanceCreationPositioningClass' 'defencePressure' 'defencePressureClass'
 'defenceAggression' 'defenceAggressionClass' 'defenceTeamWidth'
 'defenceTeamWidthClass' 'defenceDefenderLineClass'] 	 (1458, 25)


In [140]:
def get_match_outcome(df: pd.DataFrame):
    """
    This function takes a dataframe and returns a dataframe with a new column
    called 'match_outcome' which is the outcome of the match.

    Roadmap
    -------
    1. Create a new column called 'match_outcome' and set it to 0
    2. Set the value of 'match_outcome' to 1 if the home_team_goal is greater than the away_team_goal
    3. Set the value of 'match_outcome' to -1 if the home_team_goal is less than the away_team_goal
    4. If the home_team_goal is equal to the away_team_goal or there is no goal scored, the value of 'match_outcome' is already 0

    Parameters
    ----------
    df : pd.DataFrame
        The dataframe containing the match data
        
    Returns
    -------
    pd.DataFrame
        The dataframe with the new column 'match_outcome'
    """
    # create a new column called 'match_outcome' and set it to 0
    # set the value of 'match_outcome' to 1 if the home_team_goal is greater than the away_team_goal
    # set the value of 'match_outcome' to -1 if the home_team_goal is less than the away_team_goal
    if df['home_team_goal'] > df['away_team_goal']:
        df['match_outcome'] = 1
    elif df['home_team_goal'] < df['away_team_goal']:
        df['match_outcome'] = -1
    else:
        df['match_outcome'] = 0
    # if the home_team_goal is equal to the away_team_goal or there is no goal scored, the value of 'match_outcome' is already 0
    return df['match_outcome']

In [141]:
def get_player_features(df: pd.DataFrame, player_attributes: pd.DataFrame):
    """
    This function takes a dataframe row and a dataframe with player attributes 
    and returns a dataframe with the player features.

    Roadmap
    -------
    1. Get the match id, home team id, away team id, league id and date
    2. Get the result of the match
    3. Create an empty dataframe to store the player features
    4. For each player in the home team, get the player attributes for the last match
    5. For each player in the away team, get the player attributes for the last match
    6. Return the dataframe with the player features

    Parameters
    ----------
    df : pd.DataFrame
        A dataframe row with the match information
    player_attributes : pd.DataFrame
        A dataframe with the player attributes

    Returns
    -------
    pd.DataFrame
        A dataframe with the player features
    """
    # get the match id, home team id, away team id, league id and date
    match_id = df['match_api_id']
    home_team_id = df['home_team_api_id']
    away_team_id = df['away_team_api_id']
    date = df['date']
    # get the result of the match
    match_outcome = get_match_outcome(df)

    # create an empty dataframe to store the player features
    features = pd.DataFrame()
    # for each player in the home team, get the player attributes for the last match
    for i in range(1, 2):
        # get the player id
        player_id = df[f'home_player_{i}']
        # get the player attributes for the last match
        attributes = player_attributes[player_attributes.player_api_id == player_id]
        current_attributes = attributes[attributes.date < date].sort_values(by='date', ascending=False)[:1]

        if not current_attributes.empty:
            # add the player attributes to the dataframe with suffix starting with 'home_player_{i}_'
            for col in current_attributes.columns:
                if col not in ['id', 'player_fifa_api_id', 'player_api_id', 'date']:
                    name = f'home_player_{i}_{col}'
                    value = current_attributes[col].values[0]
                    features.loc[0, name] = value
        else:
            for col in current_attributes.columns:
                if col not in ['id', 'player_fifa_api_id', 'player_api_id', 'date']:
                    name = f'home_player_{i}_{col}'
                    features.loc[0, name] = np.nan
    # for each player in the away team, get the player attributes for the last match
    for i in range(1, 2):
        # get the player id
        player_id = df[f'away_player_{i}']
        # get the player attributes for the last match
        attributes = player_attributes[player_attributes.player_api_id == player_id]
        current_attributes = attributes[attributes.date < date].sort_values(by='date', ascending=False)[:1]
        if not current_attributes.empty:
            # add the player attributes to the dataframe with suffix starting with 'away_player_{i}_'
            for col in current_attributes.columns:
                if col not in ['id', 'player_fifa_api_id', 'player_api_id', 'date']:
                    name = f'away_player_{i}_{col}'
                    value = current_attributes[col].values[0]
                    features.loc[0, name] = value
        else:
            for col in current_attributes.columns:
                if col not in ['id', 'player_fifa_api_id', 'player_api_id', 'date']:
                    name = f'away_player_{i}_{col}'
                    features.loc[0, name] = np.nan

    # collect all the attributes in a dataframe with the match id, home team id, away team id, league id, date and match outcome
    features.loc[0, 'match_api_id'] = match_id
    features.loc[0, 'home_team_api_id'] = home_team_id
    features.loc[0, 'away_team_api_id'] = away_team_id
    features.loc[0, 'league_id'] = df['league_id']
    features.loc[0, 'date'] = date
    features.loc[0, 'match_outcome'] = match_outcome

    features.reset_index(drop=True, inplace=True)

    return features.iloc[0]

In [142]:
def get_team_features(df: pd.DataFrame, team_attributes: pd.DataFrame):
    """
    This function takes a dataframe row and a dataframe with team attributes 
    and returns a dataframe with the team features.

    Roadmap
    -------
    1. Get the match id, home team id, away team id, league id and date
    2. Get the result of the match
    3. Create an empty dataframe to store the team features
    4. Get the home team attributes for the last match
    5. Get the away team attributes for the last match
    6. Return the dataframe with the team features
    
    Parameters
    ----------
    df : pd.DataFrame
        A dataframe row with the match information
    team_attributes : pd.DataFrame
        A dataframe with the team attributes

    Returns
    -------
    pd.DataFrame
        A dataframe with the team features
    """
    # get the match id, home team id, away team id, league id and date
    match_id = df['match_api_id']
    home_team_id = df['home_team_api_id']
    away_team_id = df['away_team_api_id']
    league_id = df['league_id']
    date = df['date']

    # get the result of the match
    match_outcome = get_match_outcome(df)

    # create an empty dataframe to store the team features
    features = pd.DataFrame()
    # get the home team attributes for the last match
    home_attributes = team_attributes[team_attributes.team_api_id == home_team_id]
    home_current_attributes = home_attributes[home_attributes.date < date].sort_values(by='date', ascending=False)[:1]
    if not home_current_attributes.empty:
        # add the team attributes to the dataframe with suffix starting with 'home_team_'
        for col in home_current_attributes.columns:
            if col not in ['id', 'team_fifa_api_id', 'team_api_id', 'date']:
                name = f'home_team_{col}'
                value = home_current_attributes[col].values[0]
                features.loc[0, name] = value
    else:
        for col in home_current_attributes.columns:
            if col not in ['id', 'team_fifa_api_id', 'team_api_id', 'date']:
                name = f'home_team_{col}'
                features.loc[0, name] = np.nan

    # get the away team attributes for the last match
    away_attributes = team_attributes[team_attributes.team_api_id == away_team_id]
    away_current_attributes = away_attributes[away_attributes.date < date].sort_values(by='date', ascending=False)[:1]
    if not away_current_attributes.empty:
        # add the team attributes to the dataframe with suffix starting with 'away_team_'
        for col in away_current_attributes.columns:
            if col not in ['id', 'team_fifa_api_id', 'team_api_id', 'date']:
                name = f'away_team_{col}'
                value = away_current_attributes[col].values[0]
                features.loc[0, name] = value
    else:
        for col in away_current_attributes.columns:
            if col not in ['id', 'team_fifa_api_id', 'team_api_id', 'date']:
                name = f'away_team_{col}'
                features.loc[0, name] = np.nan

    # collect all the attributes in a dataframe with the match id, home team id, away team id, league id, date and match outcome
    features.loc[0, 'match_api_id'] = match_id
    features.loc[0, 'home_team_api_id'] = home_team_id
    features.loc[0, 'away_team_api_id'] = away_team_id
    features.loc[0, 'league_id'] = league_id
    features.loc[0, 'date'] = date
    features.loc[0, 'match_outcome'] = match_outcome

    features.reset_index(drop=True, inplace=True)
    
    return features.iloc[0]

In [143]:
def get_odds_features(df: pd.DataFrame):
    """
    This function takes a dataframe row and returns a dataframe with the odds features.

    Roadmap
    -------
    1. Get the match id, home team id, away team id, league id and date.
    2. Get the result of the match.
    3. Create an empty dataframe to store the odds features.
    4. Add the odds attributes to the dataframe.
    5. Collect all the attributes in a dataframe with the match id, home team id, away team id, league id, date and match outcome.
    6. Return the dataframe.

    Parameters
    ----------
    df : pd.DataFrame
        A dataframe row with the match details.

    Returns
    -------
    pd.DataFrame
        A dataframe with the odds features.
    """
    # get the match id, home team id, away team id, league id and date
    match_id = df['match_api_id']
    home_team_id = df['home_team_api_id']
    away_team_id = df['away_team_api_id']
    league_id = df['league_id']
    date = df['date']

    # get the result of the match
    match_outcome = get_match_outcome(df)

    # create an empty dataframe to store the odds features
    features = pd.DataFrame()
    # add the odds attributes to the dataframe
    for col in df.index:
        if col.startswith('B365') or col.startswith('BW') or col.startswith('IW') or col.startswith('LB') or col.startswith('PS') or col.startswith('WH') or col.startswith('SJ') or col.startswith('VC') or col.startswith('GB') or col.startswith('BS'):
            name = col
            value = df[col]
            features.loc[0, name] = value

    # collect all the attributes in a dataframe with the match id, home team id, away team id, league id, date and match outcome
    features.loc[0, 'match_api_id'] = match_id
    features.loc[0, 'home_team_api_id'] = home_team_id
    features.loc[0, 'away_team_api_id'] = away_team_id
    features.loc[0, 'league_id'] = league_id
    features.loc[0, 'date'] = date
    features.loc[0, 'match_outcome'] = match_outcome

    features.reset_index(drop=True, inplace=True)

    return features.iloc[0]

In [144]:
# take only 1% rows of data for run time purposes
matches = matches.sample(frac=run, random_state=Seed)
matches.shape

(520, 115)

In [145]:
# get betting odds for the matches
mf_betting = matches.apply(get_odds_features, axis=1)
mf_betting.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 520 entries, 13049 to 14767
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   B365H             457 non-null    float64
 1   B365D             457 non-null    float64
 2   B365A             457 non-null    float64
 3   BWH               457 non-null    float64
 4   BWD               457 non-null    float64
 5   BWA               457 non-null    float64
 6   IWH               456 non-null    float64
 7   IWD               456 non-null    float64
 8   IWA               456 non-null    float64
 9   LBH               455 non-null    float64
 10  LBD               455 non-null    float64
 11  LBA               455 non-null    float64
 12  PSH               227 non-null    float64
 13  PSD               227 non-null    float64
 14  PSA               227 non-null    float64
 15  WHH               456 non-null    float64
 16  WHD               456 non-null    floa

In [146]:
# get player attributes for the matches
mf_player = matches.apply(get_player_features, axis=1, args=(player_attributes,))
mf_player.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 520 entries, 13049 to 14767
Data columns (total 82 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   home_player_1_overall_rating       500 non-null    float64
 1   home_player_1_potential            500 non-null    float64
 2   home_player_1_preferred_foot       500 non-null    object 
 3   home_player_1_attacking_work_rate  493 non-null    object 
 4   home_player_1_defensive_work_rate  500 non-null    object 
 5   home_player_1_crossing             500 non-null    float64
 6   home_player_1_finishing            500 non-null    float64
 7   home_player_1_heading_accuracy     500 non-null    float64
 8   home_player_1_short_passing        500 non-null    float64
 9   home_player_1_volleys              495 non-null    float64
 10  home_player_1_dribbling            500 non-null    float64
 11  home_player_1_curve                495 non-null    f

In [147]:
# get team attributes for the matches
mf_team = matches.apply(get_team_features, axis=1, args=(team_attributes,))
mf_team.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 520 entries, 13049 to 14767
Data columns (total 48 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   home_team_buildUpPlaySpeed                408 non-null    float64
 1   home_team_buildUpPlaySpeedClass           408 non-null    object 
 2   home_team_buildUpPlayDribbling            116 non-null    float64
 3   home_team_buildUpPlayDribblingClass       408 non-null    object 
 4   home_team_buildUpPlayPassing              408 non-null    float64
 5   home_team_buildUpPlayPassingClass         408 non-null    object 
 6   home_team_buildUpPlayPositioningClass     408 non-null    object 
 7   home_team_chanceCreationPassing           408 non-null    float64
 8   home_team_chanceCreationPassingClass      408 non-null    object 
 9   home_team_chanceCreationCrossing          408 non-null    float64
 10  home_team_chanceCreationCrossing

### Data Cleaning

Since we have already removed the irrelevant columns in the previous section, we will only handle the missing values in this section.

In [148]:
# check the distribution of the missing values in the betting odds dataframe
print('Missing values in betting odds dataframe: ' + str(mf_betting.isnull().sum().sum() / mf_betting.size * 100) + '%')
mf_betting.isnull().sum().sort_values(ascending=False)[:10]

Missing values in betting odds dataframe: 21.201923076923077%


PSH    293
PSA    293
PSD    293
GBH    236
BSA    236
BSD    236
BSH    236
GBA    236
GBD    236
SJA    176
dtype: int64

In [149]:
# check the distribution of the missing values in the player attributes dataframe
print('Missing values in player attributes dataframe: ' + str(mf_player.isnull().sum().sum() / mf_player.size * 100) + '%')
mf_player.isnull().sum().sort_values(ascending=False)[:10]

Missing values in player attributes dataframe: 3.4990619136960603%


home_player_1_attacking_work_rate    27
away_player_1_attacking_work_rate    26
home_player_1_volleys                25
home_player_1_jumping                25
home_player_1_sliding_tackle         25
home_player_1_balance                25
home_player_1_agility                25
home_player_1_curve                  25
home_player_1_vision                 25
away_player_1_curve                  22
dtype: int64

In [150]:
# check the distribution of the missing values in the team attributes dataframe
print('Missing values in team attributes dataframe: ' + str(mf_team.isnull().sum().sum() / mf_team.size * 100) + '%')
mf_team.isnull().sum().sort_values(ascending=False)[:10]

Missing values in team attributes dataframe: 20.78125%


home_team_buildUpPlayDribbling           404
away_team_buildUpPlayDribbling           403
home_team_buildUpPlaySpeed               112
home_team_chanceCreationShootingClass    112
home_team_buildUpPlaySpeedClass          112
home_team_defenceDefenderLineClass       112
home_team_defenceTeamWidthClass          112
home_team_defenceTeamWidth               112
home_team_defenceAggressionClass         112
home_team_defencePressureClass           112
dtype: int64

### Data Transformation

### Data Integration

### Feature Engineering

## Data Analysis

In this section, we will perform initial exploration of the data to gain a better understanding of the variables and their distributions. 
This may include visualizing the data and calculating summary statistics.

### Exploratory Data Analysis

### Descriptive Statistics

### Data Visualization

### Identifying Patterns and Insights

## Prediction

In this section, we will use machine learning techniques to predict the match outcome using the team and player attributes. We will compare the performance of different models and evaluate their accuracy.

### Model Definition

In this section, we will define the models that will be used for the prediction. We will use the following models:

- Logistic Regression
- Decision Tree
- Random Forest
- Gradient Boosting
- Support Vector Machine

### Model Training

In this section, we will train the models using the training data.

### Model Testing

In this section, we will test the models using the test data.

### Model Evaluation

In this section, we will evaluate the performance of the models using the test data.

### Model Selection

In this section, we will select the best model based on the evaluation results.

### Model Tuning

In this section, we will tune the selected model to improve its performance.

## Conclusion

### Discussion

### Limitations

### Future Work